# 03 — Deep CNN with Pooling

`Conv2D(1→4) → ReLU → MaxPool(2×2) → Conv2D(4→8) → ReLU → Flatten → Dense(72→10)`

In [1]:
from _setup import setup_all, PROJECT_ROOT
setup_all()

   NANO-RUST Setup
✅ Virtualenv: C:\Users\suoya\OneDrive\Documents\nanorust\.venv
✅ numpy
✅ torch
✅ maturin

✅ rustc 1.93.1 (01f6ddf75 2026-02-11)

🔧 CARGO_TARGET_DIR = C:\Users\suoya\.nanorust_target
🔨 Building nano-rust-py (maturin develop --release)...
✅ Build OK (1.9s)
       Finished `release` profile [optimized] target(s) in 0.02s
   📦 Built wheel for CPython 3.13 to C:\Users\suoya\AppData\Local\Temp\.tmpjjZxK5\nano_rust_py-0.1.0-cp313-cp313-win_amd64.whl

🎉 Setup complete! Ready to use.


In [2]:
import numpy as np
import torch
import torch.nn as nn
from nano_rust_utils import quantize_to_i8, quantize_weights, calibrate_model
import nano_rust_py

torch.manual_seed(42)
model = nn.Sequential(
    nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=0),   # [1,12,12]→[4,10,10]
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),                  # [4,10,10]→[4,5,5]
    nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=0),   # [4,5,5]→[8,3,3]
    nn.ReLU(),
    nn.Flatten(),                                           # →[72]
    nn.Linear(72, 10, bias=True),
)
model.eval()
print(model)

q_weights = quantize_weights(model)

torch.manual_seed(123)
input_tensor = torch.randn(1, 1, 12, 12)
q_input, input_scale = quantize_to_i8(input_tensor.numpy().flatten())

requant = calibrate_model(model, input_tensor, q_weights, input_scale)
print('Requant:', {k: (m, s) for k, (m, s, _) in requant.items()})

Sequential(
  (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (4): ReLU()
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=72, out_features=10, bias=True)
)
Requant: {'0': (8617055, 31), '3': (4918802, 31), '6': (15159516, 31)}


In [3]:
with torch.no_grad():
    pytorch_out = model(input_tensor).numpy().flatten()

nano = nano_rust_py.PySequentialModel(input_shape=[1, 12, 12], arena_size=65536)

m0, s0, b0 = requant['0']
nano.add_conv2d_with_requant(
    q_weights['0']['weights'].flatten().tolist(), b0,
    1, 4, 3, 3, 1, 0, m0, s0)
nano.add_relu()
nano.add_max_pool2d(2, 2, 2)

m3, s3, b3 = requant['3']
nano.add_conv2d_with_requant(
    q_weights['3']['weights'].flatten().tolist(), b3,
    4, 8, 3, 3, 1, 0, m3, s3)
nano.add_relu()
nano.add_flatten()

m6, s6, b6 = requant['6']
nano.add_dense_with_requant(
    q_weights['6']['weights'].flatten().tolist(), b6, m6, s6)

nano_out = nano.forward(q_input.tolist())

q_pytorch, _ = quantize_to_i8(pytorch_out)
nano_arr = np.array(nano_out, dtype=np.int8)
diff = np.abs(q_pytorch.astype(np.int32) - nano_arr.astype(np.int32))

print(f'PyTorch (i8):  {q_pytorch.tolist()}')
print(f'NANO-RUST:     {nano_arr.tolist()}')
print(f'Max diff: {np.max(diff)}, Mean diff: {np.mean(diff):.2f}')
print(f'Class: PyTorch={np.argmax(q_pytorch)}, NANO-RUST={np.argmax(nano_arr)}')
print(f'\n{"✅ PASS" if np.max(diff) <= 20 else "❌ FAIL"} (tolerance=20)')

PyTorch (i8):  [-6, -3, -44, -100, -115, 66, 127, 115, -66, -9]
NANO-RUST:     [-8, -4, -39, -101, -117, 67, 127, 117, -67, -9]
Max diff: 5, Mean diff: 1.50
Class: PyTorch=6, NANO-RUST=6

✅ PASS (tolerance=20)
